# Clean Old Data 2016

In [1]:
import sys
sys.path.insert(0, '../../data/lib/')

import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import CSV

In [2]:
types = {
    'name': 'str',
    'location': 'str',
    'country': 'str',
    'address': 'str',
    'uci': 'str',
    'type': 'str',
    'source': 'str'
}

# Read pdf into DataFrame
#df = pd.read_csv("../../data/0. old data/1. transformation/2015_list.csv", dtype = types)
df_2016 = pd.read_excel("../../data/0. old data/1. transformation/0_2016_list.xlsx", dtype = types)

columns_final = ['name', 'location', 'country', 'address', 'plz', 'uci', 'donations_grants',
   'sponsorship', 'registration_fees', 'travel_accommodation', 'fees',
   'related_expenses', 'total', 'type', 'source', 'year']

## Format Table

In [3]:
def revert_name_source(dataset, sep, source):
    for index, row in dataset[(dataset.type == 'hcp') & (dataset.source.isin(source))].iterrows():
        if row['type'] == 'hcp':
            splitted = row['name'].strip().split(sep)
            if len(splitted) == 2:
                dataset.loc[index, 'name'] =  (' '.join(splitted[::-1])).strip()
            else:
                if len(splitted) == 3:
                    dataset.loc[index, 'name'] =  splitted[1] + ' ' + splitted[2] + ' ' + splitted[0]
                else:
                    dataset.loc[index, 'name'] =  (' '.join(splitted[::-1])).strip()
                #dataset.loc[index, '_export_information'] = 'Name besteht aus mehr als drei Wörtern. Evt falsch gedrecht. Bitte checken. Original: ' + row['name']
    
    return dataset

In [4]:
df_export = df_2016.copy()

#Rename
df_export = df_export.rename(columns={'company': 'source'})

#Remove rows which have no values
df_export = df_export.replace("nan", np.nan)
df_export = df_export.dropna(subset=['name'], how='all')

#Add Missing fields
df_export['country'] = 'CH'
df_export['uci'] = ''
df_export['plz'] = ''
df_export['year'] = 2016

#lowercase
df_export['type'] = df_export['type'].str.lower()

#Titlez actelion
df_export.loc[df_export.source.isin(['amgen']), 'name'] = df_export['name'].str.title()
df_export.loc[df_export.source.isin(['actelion', 'amgen']), 'location'] = df_export['location'].str.title()
df_export.loc[df_export.source.isin(['amgen']), 'address'] = df_export['address'].str.title()

#Carrination
df_export = remove_carination(df_export, ' ')

#basic string conversion
df_export = basic_string_conversion(df_export)

#Remove text in fields
df_export.loc[df_export.address == 'nan', 'address'] = np.nan
df_export.loc[df_export.location == 'nan', 'location'] = np.nan
df_export['name'] = df_export['name'].str.replace('  ', ' ')
df_export['name'] = df_export['name'].str.replace(', No Title', '', regex=False)
df_export['name'] = df_export['name'].str.replace(', pract.', '', regex=False)
df_export['name'] = df_export['name'].str.replace(', lic.', '', regex=False)
df_export['name'] = df_export['name'].str.replace('Egregio Dott.', '', regex=False)
df_export['name'] = df_export['name'].str.replace('Egr gio Dott.', '', regex=False)

#Remove Titles in Name
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Remove Comma at the end
df_export['name'] = df_export['name'].str.replace(r',$', '', regex=True)

#Remove comma in otsuka
df_export.loc[df_export.source == 'otsuka', 'name'] = df_export['name'].str.replace(', ', ' ')

#Remove "r."
df_export['name'] = df_export['name'].str.replace(r'\br. ', '', regex=True)

#shift lastname - surname
df_export = revert_name_source(df_export, ' ', ['otsuka'])
df_export = revert_name_source(df_export, ', ', ['abbvie', 'actelion', 'amgen', 'astellas', 'astrazeneca',
                                                 'baxter', 'bayer', 'mepha', 'pfizer', 'teva', 'bgp', 'biogen', 'boehringer',
                                                 'bristol', 'celgene', 'csl', 'ferring', 'future', 'galderma', 'gilead',
                                                 'glaxosmithkline', 'gruenenthal', 'janssen', 'leo', 'lundbeck', 'menarini',
                                                 'merck', 'msd', 'mundipharma', 'novartis', 'novo', 'orion', 'pierre',
                                                 'roche', 'sandoz', 'sanofi', 'servier', 'stallergenes', 'takeda', 'ucb',
                                                 'vifor', 'zambon'])

#Convert to Numbers
df_export = amounts_to_number(df_export)

#Reorder Columns
df_export = df_export[columns_final]

#CHECK DATAFRAME
check_dataframe_list(df_export)

#save
df_export.to_csv('../../data/0. old data/1. transformation/1_2016_list_cleaned.csv', index=False)

#write_to_excel(df_export, 'tmp.xlsx', open=True)


Column 'year' zuviel
Duplicates found. Please check for duplicates: df_export[df_export.duplicated()]
Total nicht Summe der Werte
